In [1]:
import os
import cv2

def convert_images_to_grayscale(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all files in the input folder
    image_files = os.listdir(input_folder)

    for image_file in image_files:
        # Create the input and output file paths
        input_path = os.path.join(input_folder, image_file)
        output_path = os.path.join(output_folder, image_file)

        # Read the image in RGB format
        image = cv2.imread(input_path)

        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Save the grayscale image
        cv2.imwrite(output_path, gray_image)

if __name__ == "__main__":
    # Specify the input and output folders
    input_folder = r"D:\downloads\Dataset_extracted_from_original\Scratch"
    output_folder = r"D:\downloads\Dataset_extracted_from_original\scratch2"

    # Call the function to convert images to grayscale
    convert_images_to_grayscale(input_folder, output_folder)



In [2]:
import numpy as np
from skimage.feature import graycomatrix
#from skimage.feature import greycomatrix, greycoprops
from skimage.feature import graycoprops
from skimage.filters import gabor
from skimage import color, io
import cv2

In [3]:
def extract_glcm_features(image):
    
#     # Convert to RGB if necessary
#     if image.shape[-1] == 4:
#         image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        
#     # Convert the image to grayscale
#     gray_image = color.rgb2gray(image)
#     cv2.imshow('RGB Image', image)
#     cv2.imshow('Grayscale Image', gray_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
        # Compute GLCM properties
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = graycomatrix(image, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
    glcm_features = [graycoprops(glcm, prop).ravel() for prop in glcm_props]
    return np.concatenate(glcm_features)

In [5]:
def extract_gabor_features(image):
    # Convert the image to grayscale
#     gray_image = color.rgb2gray(image)

    # Define Gabor filter parameters
    frequencies = [0.1, 0.5]
    thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]

    # Compute Gabor filter responses
    gabor_features = []
    for freq in frequencies:
        for theta in thetas:
            filt_real, filt_imag = gabor(image, frequency=freq, theta=theta)
            gabor_features.append(np.mean(filt_real))
            gabor_features.append(np.std(filt_real))
            gabor_features.append(np.mean(filt_imag))
            gabor_features.append(np.std(filt_imag))

    return np.array(gabor_features)


In [6]:
# Path to the image file
image_path = r'D:\downloads\Dataset_extracted_from_original\scratch2\1.jpg'

# Load the image
image = io.imread(image_path)

# Extract GLCM features
glcm_feats = extract_glcm_features(image)

print(glcm_feats)



[43.18161387 83.39718368 44.86841059 81.42215295  2.58680895  3.62148582
  2.59619618  3.54648611  0.65784826  0.62153607  0.67090408  0.62195208
  0.31806565  0.31100992  0.31959991  0.31106608  0.99015129  0.98099749
  0.98976657  0.98144751]


In [7]:
# Path to the image file
image_path = r'D:\downloads\Dataset_extracted_from_original\scratch2\1.jpg'

# Load the image
image = io.imread(image_path)



# Extract Gabor filter features
gabor_feats = extract_gabor_features(image)

print(gabor_feats)


# print(f"Concatenated feature vector length: {len(concatenated_features)}")

[17.79258728 64.43963142 19.04263687 66.53385558 19.35662079 66.9439956
 20.13740158 68.19604741 19.80325317 67.55356701 20.66819763 68.91917827
 18.58610535 65.71951517 19.13625717 66.62882066  4.16798782 32.05974208
  0.          0.          0.58850861 12.14654741  0.97317123 15.68512022
  2.39437485 24.35558883  0.          0.          0.52170563 11.43210103
  0.83732986 14.54873075]


In [8]:
# Concatenate GLCM and Gabor features
concatenated_features = np.concatenate((glcm_feats, gabor_feats))

print(f"Concatenated feature vector length: {len(concatenated_features)}")
print(concatenated_features)

Concatenated feature vector length: 52
[43.18161387 83.39718368 44.86841059 81.42215295  2.58680895  3.62148582
  2.59619618  3.54648611  0.65784826  0.62153607  0.67090408  0.62195208
  0.31806565  0.31100992  0.31959991  0.31106608  0.99015129  0.98099749
  0.98976657  0.98144751 17.79258728 64.43963142 19.04263687 66.53385558
 19.35662079 66.9439956  20.13740158 68.19604741 19.80325317 67.55356701
 20.66819763 68.91917827 18.58610535 65.71951517 19.13625717 66.62882066
  4.16798782 32.05974208  0.          0.          0.58850861 12.14654741
  0.97317123 15.68512022  2.39437485 24.35558883  0.          0.
  0.52170563 11.43210103  0.83732986 14.54873075]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Assuming you have your concatenated features stored in X and labels in y
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(concatenated_features, labels, test_size=0.2, random_state=42)

# Initialize SVM classifier
svm_classifier = SVC(kernel='linear')

# Train the classifier
svm_classifier.fit(X_train, y_train)

# Predict on test data
predictions = svm_classifier.predict(X_test)

# Evaluate the classifier (e.g., accuracy, confusion matrix)
# ...


In [ ]:
import numpy as np
from skimage.feature import graycomatrix
#from skimage.feature import greycomatrix, greycoprops
from skimage.feature import graycoprops
from skimage.filters import gabor
from skimage import color, io
import cv2

def extract_glcm_features(image):
    
    # Convert to RGB if necessary
    if image.shape[-1] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        
    # Convert the image to grayscale
    gray_image = color.rgb2gray(image)
    cv2.imshow('RGB Image', image)
    cv2.imshow('Grayscale Image', gray_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Compute GLCM properties
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = greycomatrix(gray_image, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
    glcm_features = [greycoprops(glcm, prop).ravel() for prop in glcm_props]
    return np.concatenate(glcm_features)

def extract_gabor_features(image):
    # Convert the image to grayscale
    gray_image = color.rgb2gray(image)

    # Define Gabor filter parameters
    frequencies = [0.1, 0.5]
    thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]

    # Compute Gabor filter responses
    gabor_features = []
    for freq in frequencies:
        for theta in thetas:
            filt_real, filt_imag = gabor(gray_image, frequency=freq, theta=theta)
            gabor_features.append(np.mean(filt_real))
            gabor_features.append(np.std(filt_real))
            gabor_features.append(np.mean(filt_imag))
            gabor_features.append(np.std(filt_imag))

    return np.array(gabor_features)

# Path to the image file
image_path = '/media/rashmi/Personal/Pranshu/fina_aug_dataset/valid/glioma/15.png'

# Load the image
image = io.imread(image_path)

# Extract GLCM features
glcm_feats = extract_glcm_features(image)

# Extract Gabor filter features
# omatrix, greycopropsgabor_feats = extract_gabor_features(image)

# Concatenate GLCM and Gabor features
# concatenated_features = np.concatenate((glcm_feats, gabor_feats))

print(f"Concatenated feature vector length: {len(concatenated_features)}")

In [ ]:
the input array must have size 3 along `channel_axis`, got (512, 512, 4)